In [1]:
import pandas as pd
import requests
from io import BytesIO
import os
import xlrd
import time



In [2]:
def adjclose(code):
    url = 'https://query1.finance.yahoo.com/v7/finance/download/%s?period1=1167231600&period2=1493737200&interval=1d&events=history&crumb=OPrK1lH94iY'%(code)
    data = {
            'period1':'1167231600',
            'period2':'1493737200',
            'interval':'1d',
            'events':'history',
            'crumb':'2g21JVm4.C2'
        }
    r= requests.post(url, data)
    Date=[]
    Open=[]
    High=[]
    Low=[]
    Close=[]
    AdjClose = []
    Volume=[]
    i=0
    for row in r.text.split('\n')[1:]:
        for columns in row.split(','):
            i += 1
            if i % 7 == 1:
                Date.append(columns)
            elif i % 7 == 2:
                Open.append(columns)
            elif i % 7 == 3:
                High.append(columns)
            elif i % 7 == 4:
                Low.append(columns)
            elif i % 7 == 5:
                Close.append(columns)
            elif i % 7 == 6:
                AdjClose.append(columns)
            else:
                Volume.append(columns)
    Date=Date[0:-1]
    df = pd.DataFrame({'Date':Date,'Open':Open,'High':High,'Low':Low,'Close':Close,code:AdjClose,'Volume':Volume}, columns=['Open','High','Low','Close',code,'Volume'], index=Date)
    dfa = df.loc[: , [code]]
    return dfa
wb = xlrd.open_workbook("NYSE.xlsx")
ws = wb.sheet_by_index(0)
ncol = ws.ncols
nlow = ws.nrows

n = ws.col_values(0)[1:]
dfb=adjclose('IBM')
j=0


FileNotFoundError: [Errno 2] No such file or directory: 'NYSE.xlsx'

In [ ]:
for j in n:
    print(j)
    dfb[j]=adjclose(j)[j]
        
dfb

In [ ]:
dfb

In [ ]:
dfc=dfb.T

In [ ]:
dfc.to_csv('nyse_price2.csv')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import re
import json
import requests
from datetime import datetime
import pandas as pd
from pandas.io.json import json_normalize
import xlrd

In [ ]:
def get_daily_per_morningstar(symbol='XNYS:IBM'):
    # sessionKey 값
    stock_ticker = symbol.split(':')[1]
    url = 'http://quotes.morningstar.com/chart/stock/chart.action?t=' + stock_ticker
    r = requests.get(url)
    m = re.search('var sessionKey = \"([0-9A-F]+)\";', r.text)
    if m == None:
        return 'a'
    session_key = m.groups()[0]
    session_key

    # tickers (PER 티커)
    url = "http://quotespeed.morningstar.com/getids.jsp?instid=MSRT&sdkver=2.4.4&symbol={}&qs_wsid={}".format(symbol, session_key)
    r = requests.get(url)
    jo = json.loads(r.text) 
    if len(jo['Records']) == 1:
        per_ticker = jo['Records'][0]['PID']
        per_ticker
    else:
        return 'a'

    str_today = datetime.today().strftime('%Y%m%d') # 오늘 날짜

    # PER URL 생성 (daily, 10년) 
    url_tmpl = 'http://quotespeed.morningstar.com/ra/uniqueChartData?'
    url_tmpl += 'f=d&country=USA&fields=HS0A2&sd=20070101&ed={}&tickers={}&cdt=2&instid=MSRT&sdkver=2.4.4&qs_wsid={}'
    url = url_tmpl.format(str_today, per_ticker, session_key)

    r = requests.get(url)
    jo = json.loads(r.text)

    df = json_normalize(jo['Data'][0]['DailyData']['HS0A2'])
    if len(df) == 0:
        return 'a'
    df_per = df[['Date', 'Last']].copy()
    df_per.columns = ['date', 'PER']
    df_per['PER'] = df_per['PER'].astype(float)
    df_per['date'] = pd.to_datetime(df_per['date'])
    df_per.set_index('date', inplace=True)
    df_per=df_per.rename(columns = {'PER' : symbol[5:]})

    return df_per

In [ ]:
wb = xlrd.open_workbook("NYSE.xlsx")
ws = wb.sheet_by_index(0)
ncol = ws.ncols
nlow = ws.nrows

n = ws.col_values(0)[1:]
j=0
df = get_daily_per_morningstar('XNYS:IBM')

In [ ]:
for j in n:
    if j != "ACST" and len(get_daily_per_morningstar(str('XNYS:'+j))) != 1:
        print(j)
        df[j]=get_daily_per_morningstar(str('XNYS:'+j))[j]
    else:
        print('%s is none'%j)
df


In [ ]:
dfd=df.T
dfd

In [ ]:
dfd.to_csv('nyse_per2.csv')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import re
import json
import requests
from datetime import datetime
import pandas as pd
from pandas.io.json import json_normalize
import xlrd

In [ ]:
def get_daily_pbr_morningstar(symbol='XNYS:IBM'):
    # sessionKey 값
    stock_ticker = symbol.split(':')[1]
    url = 'http://quotes.morningstar.com/chart/stock/chart.action?t=' + stock_ticker
    r = requests.get(url)
    m = re.search('var sessionKey = \"([0-9A-F]+)\";', r.text)
    if m == None:
        return 'a'
    session_key = m.groups()[0]
    session_key

    # tickers (PER 티커)
    url = "http://quotespeed.morningstar.com/getids.jsp?instid=MSRT&sdkver=2.4.4&symbol={}&qs_wsid={}".format(symbol, session_key)
    r = requests.get(url)
    jo = json.loads(r.text) 
    if len(jo['Records']) == 1:
        per_ticker = jo['Records'][0]['PID']
        per_ticker
    else:
        return 'a'

    str_today = datetime.today().strftime('%Y%m%d') # 오늘 날짜

    # PER URL 생성 (daily, 10년) 
    url_tmpl = 'http://quotespeed.morningstar.com/ra/uniqueChartData?'
    url_tmpl += 'f=d&country=USA&fields=HS0A0&sd=20070101&ed={}&tickers={}&cdt=2&instid=MSRT&sdkver=2.4.4&qs_wsid={}'
    url = url_tmpl.format(str_today, per_ticker, session_key)

    r = requests.get(url)
    jo = json.loads(r.text)

    df = json_normalize(jo['Data'][0]['DailyData']['HS0A0'])
    if len(df) == 0:
        return 'a'
    df_pbr = df[['Date', 'Last']].copy()
    df_pbr.columns = ['date', 'PBR']
    df_pbr['PBR'] = df_pbr['PBR'].astype(float)
    df_pbr['date'] = pd.to_datetime(df_pbr['date'])
    df_pbr.set_index('date', inplace=True)
    df_pbr=df_pbr.rename(columns = {'PBR' : symbol[5:]})

    return df_pbr

In [ ]:
wb = xlrd.open_workbook("NYSE.xlsx")
ws = wb.sheet_by_index(0)
ncol = ws.ncols
nlow = ws.nrows

n = ws.col_values(0)[1:]
j=0
df = get_daily_pbr_morningstar('XNYS:IBM')

In [ ]:
for j in n:
    if len(get_daily_pbr_morningstar(str('XNYS:'+j))) != 1:
        print(j)
        df[j]=get_daily_pbr_morningstar(str('XNYS:'+j))[j]
    else:
        print('%s is none'%j)
df

In [ ]:
df

In [ ]:
dfa=df.T
dfa.to_csv('nyse_pbr.csv')